https://qiita.com/cvusk/items/1f3b178f34c39beb29ff#_reference-3c5df6e5237c0c705690
https://github.com/shibuiwilliam/keras_opt/blob/master/tpe_nn.ipynb


In [1]:
from hyperopt import hp, tpe, Trials, fmin
import numpy as np
import pandas as pds
import random
from keras.layers import Activation, Dropout, BatchNormalization, Dense
from keras.models import Sequential
from keras.datasets import mnist
from keras.metrics import categorical_crossentropy
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# MNIST class
class MNIST():
    def __init__(self,
                 l1_out=512, 
                 l2_out=512, 
                 l1_drop=0.2, 
                 l2_drop=0.2, 
                 bn1=0,
                 bn2=0,
                 batch_size=100, 
                 epochs=10, 
                 validation_split=0.1):
        self.l1_out = l1_out
        self.l2_out = l2_out
        self.l1_drop = l1_drop
        self.l2_drop = l2_drop
        self.bn1 = bn1
        self.bn2 = bn2
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.mnist_data()
        self.__model = self.mnist_model()
        params = """
        validation_split:\t{0}
        l1_drop:\t{1}
        l2_drop:\t{2}
        l1_out:\t{3}
        l2_out:\t{4}
        bn1:\t{5}
        bn2:\t{6}
        batch_size:\t{7}
        epochs:\t{8}
        """.format(self.validation_split,
                   self.l1_drop, self.l2_drop,
                   self.l1_out, self.l2_out,
                   self.bn1, self.bn2,
                   self.batch_size, self.epochs)
        print(params)
        
    # load mnist data from keras dataset
    def mnist_data(self):
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        X_train = X_train.reshape(60000, 784)
        X_test = X_test.reshape(10000, 784)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255

        Y_train = np_utils.to_categorical(y_train, 10)
        Y_test = np_utils.to_categorical(y_test, 10)
        return X_train, X_test, Y_train, Y_test
    
    # mnist model
    def mnist_model(self):
        model = Sequential()
        model.add(Dense(self.l1_out, input_shape=(784,)))
        if self.bn1 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l1_drop))
        model.add(Dense(self.l2_out))
        if self.bn2 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l2_drop))
        model.add(Dense(10))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model
    
    # fit mnist model
    def mnist_fit(self):
        early_stopping = EarlyStopping(patience=0, verbose=1)
        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split,
                       callbacks=[early_stopping])
    
    # evaluate mnist model
    def mnist_evaluate(self):
        self.mnist_fit()
        
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation

In [3]:
# function to run mnist class
def run_mnist(args):
    _mnist = MNIST(**args)
    mnist_evaluation = _mnist.mnist_evaluate()
    print("loss:{0} \t\t accuracy:{1}".format(mnist_evaluation[0], mnist_evaluation[1]))
    return mnist_evaluation[0]

In [4]:
hyperopt_parameters = {
    'validation_split': hp.uniform('validation_split', 0.0, 0.3),
    'l1_drop': hp.uniform('l1_drop', 0.0, 0.3),
    'l2_drop': hp.uniform('l2_drop', 0.0, 0.3),
    'l1_out': hp.choice('l1_out', [64, 128, 256, 512, 1024]),
    'l2_out': hp.choice('l2_out', [64, 128, 256, 512, 1024]),
    'bn1': hp.choice('bn1', [0, 1]),
    'bn2': hp.choice('bn2', [0, 1]),
    'batch_size': hp.choice('batch_size', [10, 100, 500]),
    'epochs': hp.choice('epochs', [5, 10, 20]),
}

In [5]:
#max_evals = 20
max_evals = 5

trials = Trials()

best = fmin(run_mnist, hyperopt_parameters, 
            algo=tpe.suggest, max_evals=max_evals,
           trials=trials, verbose=1)


        validation_split:	0.0470567903005738
        l1_drop:	0.243413752406138
        l2_drop:	0.15153456562089088
        l1_out:	256
        l2_out:	1024
        bn1:	1
        bn2:	1
        batch_size:	500
        epochs:	5
        
loss:0.0764614817686379 		 accuracy:0.9772000074386596

        validation_split:	0.021298429490386694
        l1_drop:	0.29916283004064353
        l2_drop:	0.26395125171105216
        l1_out:	256
        l2_out:	1024
        bn1:	1
        bn2:	1
        batch_size:	500
        epochs:	5
        
Epoch 00004: early stopping
loss:0.08083765651099384 		 accuracy:0.9746000021696091

        validation_split:	0.14375980331133842
        l1_drop:	0.17405695870197446
        l2_drop:	0.07459262432747911
        l1_out:	256
        l2_out:	64
        bn1:	1
        bn2:	1
        batch_size:	500
        epochs:	5
        
loss:0.08994058188982308 		 accuracy:0.9721000075340271

        validation_split:	0.03347764989401529
        l1_drop:	0.09752580698469

In [6]:
print(best)

{'batch_size': 2, 'bn1': 1, 'bn2': 1, 'epochs': 0, 'l1_drop': 0.243413752406138, 'l1_out': 2, 'l2_drop': 0.15153456562089088, 'l2_out': 4, 'validation_split': 0.0470567903005738}


In [8]:
trials.best_trial

{'book_time': datetime.datetime(2018, 2, 28, 1, 30, 31, 15000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'batch_size': [0],
   'bn1': [0],
   'bn2': [0],
   'epochs': [0],
   'l1_drop': [0],
   'l1_out': [0],
   'l2_drop': [0],
   'l2_out': [0],
   'validation_split': [0]},
  'tid': 0,
  'vals': {'batch_size': [2],
   'bn1': [1],
   'bn2': [1],
   'epochs': [0],
   'l1_drop': [0.243413752406138],
   'l1_out': [2],
   'l2_drop': [0.15153456562089088],
   'l2_out': [4],
   'validation_split': [0.0470567903005738]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2018, 2, 28, 1, 31, 11, 802000),
 'result': {'loss': 0.0764614817686379, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 0,
 'version': 0}

In [9]:
trials.best_trial["result"]

{'loss': 0.0764614817686379, 'status': 'ok'}

In [10]:
trials.losses()

[0.0764614817686379,
 0.08083765651099384,
 0.08994058188982308,
 0.09481895059987437,
 0.08072020234540105]

In [31]:
trials.vals

{'batch_size': [2, 2, 2, 1, 2],
 'bn1': [1, 1, 1, 1, 1],
 'bn2': [1, 1, 1, 0, 1],
 'epochs': [0, 0, 0, 0, 1],
 'l1_drop': [0.243413752406138,
  0.29916283004064353,
  0.17405695870197446,
  0.09752580698469805,
  0.02423098522754854],
 'l1_out': [2, 2, 2, 0, 4],
 'l2_drop': [0.15153456562089088,
  0.26395125171105216,
  0.07459262432747911,
  0.17428465373003163,
  0.24684197294204854],
 'l2_out': [4, 4, 0, 0, 3],
 'validation_split': [0.0470567903005738,
  0.021298429490386694,
  0.14375980331133842,
  0.03347764989401529,
  0.03802801259781645]}

In [32]:
trials.trials[0]

{'book_time': datetime.datetime(2018, 2, 28, 1, 30, 31, 15000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'batch_size': [0],
   'bn1': [0],
   'bn2': [0],
   'epochs': [0],
   'l1_drop': [0],
   'l1_out': [0],
   'l2_drop': [0],
   'l2_out': [0],
   'validation_split': [0]},
  'tid': 0,
  'vals': {'batch_size': [2],
   'bn1': [1],
   'bn2': [1],
   'epochs': [0],
   'l1_drop': [0.243413752406138],
   'l1_out': [2],
   'l2_drop': [0.15153456562089088],
   'l2_out': [4],
   'validation_split': [0.0470567903005738]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2018, 2, 28, 1, 31, 11, 802000),
 'result': {'loss': 0.0764614817686379, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 0,
 'version': 0}

In [30]:
trials.trials[0].keys()

dict_keys(['state', 'tid', 'spec', 'result', 'misc', 'exp_key', 'owner', 'version', 'book_time', 'refresh_time'])

In [36]:
for i, trial in enumerate(trials.trials):
    print("trial",i , ":",  trial["result"])

trial 0 : {'loss': 0.0764614817686379, 'status': 'ok'}
trial 1 : {'loss': 0.08083765651099384, 'status': 'ok'}
trial 2 : {'loss': 0.08994058188982308, 'status': 'ok'}
trial 3 : {'loss': 0.09481895059987437, 'status': 'ok'}
trial 4 : {'loss': 0.08072020234540105, 'status': 'ok'}


In [37]:
max_evals = 10

trials = Trials()

best = fmin(run_mnist, hyperopt_parameters, 
            algo=tpe.suggest, max_evals=max_evals,
           trials=trials, verbose=1)


        validation_split:	0.29555045194794216
        l1_drop:	0.281364876794158
        l2_drop:	0.07894411459483727
        l1_out:	256
        l2_out:	64
        bn1:	0
        bn2:	0
        batch_size:	100
        epochs:	20
        
Epoch 00007: early stopping
loss:0.07248647432526922 		 accuracy:0.9781000071763992

        validation_split:	0.017760884899331652
        l1_drop:	0.23382126580644425
        l2_drop:	0.17895377618155545
        l1_out:	1024
        l2_out:	512
        bn1:	1
        bn2:	0
        batch_size:	100
        epochs:	10
        
Epoch 00004: early stopping
loss:0.06325882731354796 		 accuracy:0.9801000094413758

        validation_split:	0.28805573547827273
        l1_drop:	0.2847781944333728
        l2_drop:	0.13911247783909114
        l1_out:	256
        l2_out:	64
        bn1:	1
        bn2:	0
        batch_size:	500
        epochs:	20
        
Epoch 00010: early stopping
loss:0.07069282783195377 		 accuracy:0.9785000085830688

        validation_sp

In [40]:
print(best)
print()
print(trials.best_trial["result"])
print()
print(trials.best_trial)
print()
print(trials.losses())
print()
print(trials.vals)
print()
for i, trial in enumerate(trials.trials):
    print("trial",i , ":",  trial["result"])


{'batch_size': 1, 'bn1': 1, 'bn2': 0, 'epochs': 1, 'l1_drop': 0.23382126580644425, 'l1_out': 4, 'l2_drop': 0.17895377618155545, 'l2_out': 3, 'validation_split': 0.017760884899331652}

{'loss': 0.06325882731354796, 'status': 'ok'}

{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': 0.06325882731354796, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'batch_size': [1], 'bn1': [1], 'bn2': [1], 'epochs': [1], 'l1_drop': [1], 'l1_out': [1], 'l2_drop': [1], 'l2_out': [1], 'validation_split': [1]}, 'vals': {'batch_size': [1], 'bn1': [1], 'bn2': [0], 'epochs': [1], 'l1_drop': [0.23382126580644425], 'l1_out': [4], 'l2_drop': [0.17895377618155545], 'l2_out': [3], 'validation_split': [0.017760884899331652]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2018, 2, 28, 1, 52, 8, 376000), 'refresh_time': datetime.datetime(2018, 2, 28, 1, 53, 57, 21000)}

[0.07248647432526922, 0.06325882731354796, 0.070

In [41]:
max_evals = 20

trials = Trials()

best = fmin(run_mnist, hyperopt_parameters, 
            algo=tpe.suggest, max_evals=max_evals,
           trials=trials, verbose=1)


print(best)
print()
print(trials.best_trial["result"])
print()
print(trials.best_trial)
print()
print(trials.losses())
print()
print(trials.vals)
print()
for i, trial in enumerate(trials.trials):
    print("trial",i , ":",  trial["result"])



        validation_split:	0.22132082959294627
        l1_drop:	0.2804237853663551
        l2_drop:	0.03444025436330409
        l1_out:	64
        l2_out:	128
        bn1:	1
        bn2:	1
        batch_size:	100
        epochs:	10
        
Epoch 00008: early stopping
loss:0.09931583708559628 		 accuracy:0.9701000040769577

        validation_split:	0.12673937847903513
        l1_drop:	0.017438698488009396
        l2_drop:	0.06825295950321873
        l1_out:	64
        l2_out:	64
        bn1:	1
        bn2:	0
        batch_size:	10
        epochs:	5
        
Epoch 00005: early stopping
loss:0.08665997516509379 		 accuracy:0.973999995648861

        validation_split:	0.10230494545611081
        l1_drop:	0.2267306959288699
        l2_drop:	0.005336405564466462
        l1_out:	64
        l2_out:	64
        bn1:	1
        bn2:	1
        batch_size:	10
        epochs:	10
        
Epoch 00007: early stopping
loss:0.09191914121038598 		 accuracy:0.972099995315075

        validation_split:	0.